In [1]:
import os
import sys
import gc
import ctypes
import numpy as np
import pandas as pd
import platform
import logging
from datetime import datetime, timedelta
from pathlib import Path
import shutil
import zipfile
import duckdb 
import warnings
import fastparquet
from tqdm import tqdm 
from typing import List, Optional, Union
import psutil
import time # For timing the execution
import re
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings('ignore')


# Optional: adjust pandas display for debugging; you can comment these out
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)


logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S')

In [2]:
def free_mem():
    """Try to return freed memory back to the OS (no-op on some platforms)."""
    try:
        libc = ctypes.CDLL(None)
        if hasattr(libc, "malloc_trim"):
            libc.malloc_trim(0)
    except Exception:
        pass

In [3]:
# --- DUCKDB CHUNK GENERATOR (Fixed) ---

def duckdb_chunk_generator(conn, sql_query, chunk_size):
    """Generates Pandas DataFrames in chunks directly from DuckDB cursor."""
    cursor = conn.cursor() 
    cursor.execute(sql_query)
    
    while True:
        # Uses the corrected method name: fetch_df_chunk
        chunk = cursor.fetch_df_chunk(chunk_size) 
        if chunk is None or chunk.empty:
            break
        yield chunk

# --- ROBUST FILE EXTRACTION (Fixed from OSErrors) ---

def extract_files_to_disk(zip_path, output_dir):
    """Cleans directory and extracts all Parquet files from ZIP."""
    if output_dir.exists():
        logging.info(f"🧹 Clearing existing directory: {output_dir.resolve()}")
        # Robust cleanup to avoid OS/lock issues
        try:
            shutil.rmtree(output_dir)
        except OSError:
             for item in output_dir.iterdir():
                if item.is_dir():
                    shutil.rmtree(item)
                else:
                    os.remove(item) 
             os.rmdir(output_dir)

    output_dir.mkdir(parents=True)
        
    logging.info("🔄 Extracting ALL Parquet files from ZIP to disk...")
    try:
        with zipfile.ZipFile(zip_path, "r") as z:
            all_files_to_extract = [f for f in z.namelist() if f.endswith(".parquet")]
            logging.info(f"🔎 Found {len(all_files_to_extract)} total Parquet files in archive.")
            for filename in all_files_to_extract:
                z.extract(filename, path=output_dir)
            return len(all_files_to_extract)
    except FileNotFoundError:
        raise FileNotFoundError(f"❌ ZIP file not found at: {zip_path}") from None

def setup_duckdb_query(output_dir, utc_start, utc_end, core_cols):
    """Sets up DuckDB connection and SQL query."""
    parquet_glob_path = str(output_dir.joinpath("**/*.parquet"))
    # Only select the columns you need for Stage 1 processing
    column_list = ", ".join([f'"{c}"' for c in core_cols])
    
    # CRITICAL: Predicate Pushdown filter on the internal 'timestamp' column
    sql_query = f"""
        SELECT {column_list}
        FROM read_parquet('{parquet_glob_path}')
        WHERE 
            "timestamp" >= '{utc_start.isoformat()}' AND 
            "timestamp" < '{utc_end.isoformat()}'
    """
    return duckdb.connect(), sql_query

In [4]:
def read_parquet_subset(parquet_path: str, start_dt: datetime, end_dt: datetime) -> pd.DataFrame:
    """
    Reads a subset of the processed Feather file using a date filter
    applied directly by DuckDB (predicate pushdown).
    
    Args:
        feather_path: Path to the processed Feather file.
        start_dt: Start datetime for the filter (inclusive).
        end_dt: End datetime for the filter (exclusive).
        
    Returns:
        A new DataFrame containing only the filtered data.
    """
    logging.info(f"Loading data subset from {start_dt} to {end_dt}...")
    
    # Use DuckDB to query the Feather file directly on disk
    con = duckdb.connect()
    
    # The SQL query filters rows on the disk file based on the 'timestamp' column.
    sql_query = f"""
        SELECT *
        FROM read_parquet('{parquet_path}')
        WHERE 
            "timestamp" >= '{start_dt.isoformat()}' AND 
            "timestamp" < '{end_dt.isoformat()}'
    """
    
    # Fetch the filtered, smaller DataFrame
    df_subset = con.execute(sql_query).fetchdf()
    con.close()
    
    logging.info(f"✅ Loaded {len(df_subset):,} rows for the requested subset.")
    return df_subset

In [5]:
# Define the 30-day filter window
filter_start_date = datetime(2025, 10, 1)
filter_end_date = datetime(2025, 10, 2) # Exclusive end date

# 1. Load the filtered subset safely
df_subset = read_parquet_subset(
    parquet_path="../../../data_points/extracted_parts/cpo_100/*.parquet",
    start_dt=filter_start_date,
    end_dt=filter_end_date
)

display(df_subset.head())

# 3. Aggressive memory cleanup after use (CRITICAL)
# del df_subset
# gc.collect()
# free_mem()

2025-11-25 16:05:54 - INFO - Loading data subset from 2025-10-01 00:00:00 to 2025-10-02 00:00:00...
2025-11-25 16:06:21 - INFO - ✅ Loaded 903,805 rows for the requested subset.


,id,timestamp,insert_timestamp,sequence,number_of_can_ids,number_of_records,percentage_of_can_ids,lowpressureoilpumpfaultcode,lowvoltagedcaclifesignal,batterycoolingstate,batterycoolanttemperature,odometerreading,brakepedalpos,ebsredwarningsignal,abs_ebsamberwarningsignal,insulation_status,insulation__value,lifesignalofinsulation,output_phase_voltage,output_phase_currant,motor_rpm,motor_torque,motor_temperature,igbt_temperature,mcuself_teststatus,side_door_panel_open_alarm,ignitionstatus,brakingsystemmallfunction_alarm,parking_brake_status,front_air_pressure,rear_air_pressure,brake_pad_worn_out_alarm,guna_dcp_temperature,guna_dcm_temperature,gunb_dcp_temperature,gunb_dcm_temperature,batterypackovervoltagealarm,monomerovervoltagealarm,polehightemperaturealarm,batteryhightemperaturealarm,hightempalarmofcharginggun,monomerundervoltagealarm,batterypackundervoltagealarm,temperaturedifferencealarm,singlevoltagedifferencealarm,chargingcurrentalarm,dischargecurrentalarm,sochighalarm,soclowalarm,bms_fault_code,gun_connection_status,maxavailableshorttermcharging,maxavailableshorttimedischarge,chargingcontactor2positive,chargingcontactor2negative,chargingcontactor1positive,batterytotalnegativecontactor,chargingcontactor1negative,pack_negative_contactors_status,bat_soc,bat_voltage,total_battery_current,bmslifesignal,steering_pump_enable_command,air_compressor_enable_command,dcdc_enable_command,vcuversioninformation,distancetoempty,accelarationpedal,brake_pedal,vcuself_teststatus,systeminterlockstate,vehiclereadycondition,vehicle_speed_vcu,gear_position,autoholdunenable,powertrain_failure_alarm,vcu_fault_code,vehiclecontrollerlife,cellmax_voltagecellnumber,cell_max_voltage,cellminvoltagecellnumber,cell_min_voltage,pack1_cellmax_temperature,pack1_maxtemperature_cell_number,pack1_cell_min_temperature,pack1_celltemperature_cellnumber,fourinonelifesignal,fourinoneself_teststatus,prechargedcontactorstate,mainmotorcontactor1state,accessorycontactorstatus,pre_charge_status,fiveinone_faultcode,airpumpcondition,air_compreesor_temperature,airpumpdcaclifesignal,dcdc_statusandfailure,dcdc_out_put_currant,dcdc_out_put_volatge,dcdcbusvoltage,dcdclifesignal,oilpumpcondition,steering_pump_rpm,steeringcontroller_failurealarm,pack_temperature1,pack_temperature2,pack_temperature3,pack_temperature4,pack_temperature5,pack_temperature6,pack_temperature7,pack_temperature8,pack_temperature9,pack_temperature10,pack_temperature11,pack_temperature12,pack_temperature13,pack_temperature14,pack_temperature15,pack_temperature16,pack_temperature17,pack_temperature18,pack_temperature19,pack_temperature20,pack_temperature21,pack_temperature22,pack_temperature23,pack_temperature24,pack_temperature25,pack_temperature26,pack_temperature27,pack_temperature28,pack_temperature29,pack_temperature30,pack_temperature31,pack_temperature32,pack_temperature33,pack_temperature34,pack_temperature35,pack_temperature36,pack_temperature37,pack_temperature38,pack_temperature39,pack_temperature40,pack_temperature41,pack_temperature42,pack_temperature43,pack_temperature44,pack_temperature45,pack_temperature46,pack_temperature47,pack_temperature48,pack_temperature49,pack_temperature50,pack_temperature51,pack_temperature52,pack_temperature53,pack_temperature54,pack_temperature55,pack_temperature56,pack_temperature57,pack_temperature58,pack_temperature59,pack_temperature60,pack_temperature61,pack_temperature62,pack_temperature63,pack_temperature64,pack_temperature65,pack_temperature66,pack_temperature67,pack_temperature68,pack_temperature69,pack_temperature70,pack_temperature71,pack_temperature72,pack_temperature73,pack_temperature74,pack_temperature75,pack_temperature76,pack_temperature77,pack_temperature78,pack_temperature79,pack_temperature80,pack_cellvoltage_1,pack_cellvoltage_2,pack_cellvoltage_3,pack_cellvoltage_4,pack_cellvoltage_5,pack_cellvoltage_6,pack_cellvoltage_7,pack_cellvoltage_8,pack_cellvoltage_9,pack_cellvoltage_10,pack_cellvoltage_11,pack_cellvoltage_12,pack_cellvoltage_13,pac

In [6]:
df_subset_small = df_subset[(df_subset.timestamp>='2025-10-01 00:00:00.030000')&(df_subset.timestamp<='2025-10-01 03:00:00.030000')]
len(df_subset_small)

111826

In [7]:
df_subset_small.groupby('id','dt')['id'].count()

ValueError: No axis named dt for object type DataFrame

In [ ]:
(df_subset.isnull().sum()[(df_subset.isnull().sum()*100/len(df_subset))>98]*100/len(df_subset)).to_csv('null_values_cpoall.csv')

In [ ]:
df_subset_small.head(100).to_csv('sample.csv')

In [ ]:
# Voltage columns: pack_cellvoltage_1 ... pack_cellvoltage_380
voltage_cols = sorted(
    [c for c in df_subset.columns if re.match(r"pack_cellvoltage_\d+", c)],
    key=lambda x: int(re.findall(r"\d+", x)[0])
)

# Temperature columns: pack_temperature_1 ... pack_temperature_80
temp_cols = sorted(
    [c for c in df_subset.columns if re.match(r"pack_temperature\d+", c)],
    key=lambda x: int(re.findall(r"\d+", x)[0])
)

print(f"Voltage columns: {len(voltage_cols)}")
print(f"Temperature columns: {len(temp_cols)}")

Voltage columns: 376
Temperature columns: 80


In [ ]:
def compute_voltage_coverage_intervals(df, voltage_cols, timestamp_col):
    """
    Computes how long it takes to receive a full set of 380 voltages 
    by accumulating non-null voltages across timestamps.
    """

    df = df.sort_values(timestamp_col).reset_index(drop=True)

    intervals = []
    seen = set()
    start_ts = None

    for idx, row in df.iterrows():
        # Start new interval if needed
        if start_ts is None:
            start_ts = row[timestamp_col]

        # Add non-null voltage columns seen in this row
        for col in voltage_cols:
            if pd.notna(row[col]):
                seen.add(col)

        # If full coverage achieved
        if len(seen) == len(voltage_cols):   # 380
            end_ts = row[timestamp_col]

            # Compute time difference
            duration_ms = end_ts - start_ts
            duration_sec = duration_ms / 1000

            intervals.append({
                "start_ts": start_ts,
                "end_ts": end_ts,
                "duration_ms": duration_ms,
                "duration_sec": duration_sec,
                "duration_rows": idx   # number of rows needed
            })

            # Reset for the next cycle
            seen = set()
            start_ts = None

    return pd.DataFrame(intervals)

non_null_cols_in_sample = df_subset[voltage_cols].columns[df_subset[voltage_cols].notna().any()].tolist()

voltage_intervals = compute_voltage_coverage_intervals(df_subset_small, voltage_cols, "timestamp")
# voltage_intervals_click = compute_voltage_coverage_intervals(df_clickhouse, voltage_cols, "timestamp_clickhouse")


plt.figure(figsize=(12,6))
sns.histplot(voltage_intervals["duration_sec"], bins=30, kde=True)
plt.title("Distribution of Full-Voltage Coverage Duration (Lakehouse)")
plt.xlabel("Seconds to receive all 380 cell voltages")
plt.ylabel("Frequency")
plt.show()

KeyboardInterrupt: 

In [ ]:
voltage_intervals.describe(percentiles=[0.25, 0.5, 0.75, 0.8,0.85,0.9,0.95, 0.99,0.995, 0.999])

In [ ]:
voltage_intervals